In [59]:
import rebound
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import math # --- setup simulation ---

In [81]:
def InitializeOrbit(imass = 0, ia = 0, ie = 0, iOmega=0.0, iomega=0.0, intf=0):
    sim = rebound.Simulation()
    sim.units = ('yr', 'AU', 'Msun')
    # Add Sun
    sim.add(m=1.0, x=0., y=0., z=0., vx=0., vy=0., vz=0.)
    
    # Add Earth
    sim.add(m=3e-6, a=1.0, e=0.0167, inc=0.0, Omega=0.0, omega=0.0, f=0.0)
    
    # Add intruder
    sim.add(m=imass, a=ia, e=ie, Omega=iOmega, omega=iomega, f=intf)
    
    return sim
def UpdateOrbit(sim, time):
    sim.integrate(time)
    sim.move_to_hel()
    return sim

def distFromSun(earth):
    return np.sqrt(earth.x*earth.x+earth.y*earth.y+earth.z*earth.z)



In [103]:
sim = InitializeOrbit(imass=1e-6, ia=-2.0, ie=1.3, iOmega=-3.0, iomega=1.0, intf = 0.5)
sim.status()
sim = UpdateOrbit(sim, 100)
sim.status()
dist = distFromSun(sim.particles[1])
if dist < 0.38 or dist > 10:
    print(f"Distance: {dist} --> Earth Ejected!")
else: 
    print(f"Distance: {dist} --> Earth not Ejected!")


---------------------------------
REBOUND version:     	4.4.10
REBOUND built on:    	Jul  1 2025 16:06:26
Number of particles: 	3
Selected integrator: 	ias15
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.001000
---------------------------------
<rebound.particle.Particle object at 0x12cdf1a40, m=1.0 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>
<rebound.particle.Particle object at 0x12cdf18c0, m=3e-06 x=0.9833000000000001 y=0.0 z=0.0 vx=0.0 vy=6.388894397887644 vz=0.0>
<rebound.particle.Particle object at 0x12cdf1a40, m=1e-06 x=0.04560026126386561 y=-0.6429868361286077 z=0.0 vx=11.657519019411628 vy=-2.5151384133188617 vz=0.0>
---------------------------------
The following fields have non-default values:
G:
< 1.000000e+00
---
> 3.947693e+01
N:
< 0
---
> 3
python_unit_l:
< 0
---
> 12937928
python_unit_m:
< 0
---
> 2145773914
python_unit_t:
< 0
---
> 1864791206
rand_seed:
< 543678
---
> 542394
particles:
> (384 bytes, values not printed)

-------------------------------

In [55]:
print(sim.units)

{'length': 'au', 'mass': 'msun', 'time': 'yr'}


In [104]:
def update_plot(time):
    # Integrate simulation to the chosen time
    sim.integrate(time)
    op = rebound.OrbitPlot(sim)
    # Update existing plot
    op.update(updateLimits=False)
    op.particles.set_color(["green", "red"])
   


# --- create interactive slider ---
interact(update_plot, time=FloatSlider(min=0, max=10, step=0.005, value=0, continuous_update=True))


interactive(children=(FloatSlider(value=0.0, description='time', max=10.0, step=0.005), Output()), _dom_classe…

<function __main__.update_plot(time)>

In [111]:
import numpy as np
import csv

# --- Define parameter ranges ---
masses = np.logspace(-4, 1, 30)   # 1e-6 → 10 (log-spaced)
a_values = [-2.0, -1.5, -1.0]
e_values = np.linspace(1.1,2,9)

# --- Fixed parameters ---
iOmega = -3.0
iomega = 1.0
intf = 0.5

# --- Precompute total runs ---
total_runs = len(masses) * len(a_values) * len(e_values)
run_counter = 0

# --- Write to CSV ---
with open('orbit_results.csv', mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['mass', 'a', 'e', 'Omega', 'omega', 'tf', 'final_dist', 'ejected'])

    for imass in masses:
        for ia in a_values:
            for ie in e_values:
                run_counter += 1

                # --- Progress ---
                print(f"Running simulation {run_counter}/{total_runs} | "
                      f"mass={imass:.2e}, a={ia}, e={ie}")

                sim = InitializeOrbit(
                    imass=imass,
                    ia=ia,
                    ie=ie,
                    iOmega=iOmega,
                    iomega=iomega,
                    intf=intf
                )

                sim = UpdateOrbit(sim, 100)
                dist = distFromSun(sim.particles[1])
                ejected = dist < 0.38 or dist > 10

                writer.writerow([imass, ia, ie, iOmega, iomega, intf, dist, ejected])

print("\n✅ All simulations complete — results saved to orbit_results.csv")


Running simulation 1/810 | mass=1.00e-04, a=-2.0, e=1.1
Running simulation 2/810 | mass=1.00e-04, a=-2.0, e=1.2125000000000001
Running simulation 3/810 | mass=1.00e-04, a=-2.0, e=1.3250000000000002
Running simulation 4/810 | mass=1.00e-04, a=-2.0, e=1.4375
Running simulation 5/810 | mass=1.00e-04, a=-2.0, e=1.55
Running simulation 6/810 | mass=1.00e-04, a=-2.0, e=1.6625
Running simulation 7/810 | mass=1.00e-04, a=-2.0, e=1.775
Running simulation 8/810 | mass=1.00e-04, a=-2.0, e=1.8875
Running simulation 9/810 | mass=1.00e-04, a=-2.0, e=2.0
Running simulation 10/810 | mass=1.00e-04, a=-1.5, e=1.1
Running simulation 11/810 | mass=1.00e-04, a=-1.5, e=1.2125000000000001
Running simulation 12/810 | mass=1.00e-04, a=-1.5, e=1.3250000000000002
Running simulation 13/810 | mass=1.00e-04, a=-1.5, e=1.4375
Running simulation 14/810 | mass=1.00e-04, a=-1.5, e=1.55
Running simulation 15/810 | mass=1.00e-04, a=-1.5, e=1.6625
Running simulation 16/810 | mass=1.00e-04, a=-1.5, e=1.775
Running simulati